In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import os
%env QPROF_DATA_PATH=/home/simonpf/Dendrite/UserAreas/Simon/gprof

env: QPROF_DATA_PATH=/home/simonpf/Dendrite/UserAreas/Simon/gprof


# Training data

The interface to the training data is realized using pytorchs Dataset
class. The data object provides access to single samples in the training
data.

In [2]:
from qprof.data import RainRates
qprof_path = os.environ["QPROF_DATA_PATH"]
path = os.path.join(qprof_path, "training_data_3.nc")
data = RainRates(path)

For the actual training, the data object needs to be wrapped in a data loader, which generates batches
from the training data.

In [3]:
import torch
from torch.utils.data import DataLoader, random_split
n_train = int(0.99 * len(data))
n_val = len(data) - n_train
training_data, validation_data = random_split(data, [n_train, n_val])
training_loader = DataLoader(training_data, batch_size = 256)
validation_loader = DataLoader(validation_data, batch_size = 256)

# QRNN training

Training the QRNN is straight forward.

In [ ]:
from qprof.models import QRNN
quantiles = [0.01, 0.05, 0.15, 0.25, 0.35, 0.45, 0.5, 0.55, 0.65, 0.75, 0.85, 0.95, 0.99]
qrnn = QRNN(13, quantiles)
qrnn.train(training_loader, validation_loader)

# Evaluation

We briefly compute the calibration on the validation set.

In [52]:
intervals, frequencies = qrnn.calibration(validation_loader)